[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/02_fastai_imgscraper.ipynb)

# Vorbereitung

Laufzeit-Typ ändern. Python-Pakete von fast.ai installieren und importieren.

In [ ]:
!pip install --upgrade fastai fastcore

In [ ]:
from fastai.vision.all import *

**Wichtig!** Wenn die folgende Zelle "False" ausgibt, ist keine GPU vorhanden -> Bitte prüfe die o. g. Schritte noch einmal und starte ggf. das Notebook neu. Bei "True" hast du eine GPU-Instanz und kannst weiter machen.

In [ ]:
torch.cuda.is_available()

# Eigenen Datensatz erstellen


Für viele verschiedene Projekte ist es möglich die benötigten Daten online zu finden und daraus eigene Datensätze zu bauen. 
Dazu installieren wir ImageScraper, eine Python-Bibliothek zum erstellen von Datensätzen über DuckDuckGo.

In [ ]:
!pip install -q jmd_imagescraper

In [ ]:
from jmd_imagescraper.core import *
from pathlib import Path

Den Pfad als `path`-Variable speichern und anschließend den Datensatz herunterladen. `duckduckgo_search()` benötigt den Destination-Pfad, ein Label, einen Suchbegriff und die Anzahl der gewünschten Ergebnisse.

In [ ]:
path = Path().cwd()/"test_images"

In [ ]:
duckduckgo_search(path, "Grizzly", "grizzly bear", max_results=100)
duckduckgo_search(path, "Black", "black bear", max_results=100)
duckduckgo_search(path, "Teddy", "teddy bear", max_results=100)

In [ ]:
files = get_image_files(path)
len(files)

# Data Block API

Anders als im ersten Beispiel erzeugen wir unser DataLoaders Objekt diesmal mit Hilfe der fast.ai Data Block API.

Bei der Erzeugung eines DataBlock werden Informationen für den DataLoader mitgegeben.

In [ ]:
bears = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 get_y=parent_label,
                 splitter=RandomSplitter(valid_pct=0.2, seed=42),
                 item_tfms=Resize(224))

`blocks` ImageBlock definiert, dass der Input ein Bild ist und CategoryBlock das Label.

`get_image_files` 

`get_y=parent_label` legt fest, dass das Label dem Namen des Ordners entspricht, in welchem Sie sich befinden.

`RandomSplitter` teilt die Daten in verschiedene Datensätze auf. Einer dient als Trainingssatz und ein weiterer zur Validierung.

`Resize` skaliert jedes Bild auf eine Größe von 224x244

Der DataLoader wird nun aus dem DataBlock erzeugt. Anschließend überprüfen wir mit `dls.vocab` unsere Labels und lassen uns einen Batch anzeigen.

In [ ]:
dls = bears.dataloaders(path)

In [ ]:
dls.vocab

In [ ]:
dls.show_batch(max_n=4, nrows=1)

Es ist gängie Praxis die Bilder zu verkleinern und in eine einheitliche Form zu bringen. Dies kann dazu führen, dass einige wichtige Detals verloren gehen. fast.ai bietet deshalb Funktionen zum skalieren der Bilder ohne dabei Informationen zu verlieren.


`RandomResizedCrop()` schneidet das Bild auf einen zufällig ausgewählten Teil zu, welcher sich in jeder Epoche ändert. Dadurch lernt das Modell, sich auf verschiedene Merkmale des Bildes zu konzentrieren und diese zu erkennen. 

In [ ]:
bears = bears.new(item_tfms=RandomResizedCrop(224, min_scale=0.5))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

# Data Augmentation

`aug_transforms()` erstellt zufällige Variationen der Eingabedaten, so dass diese unterschiedlich aussehen, aber keine wichtigen Daten verloren gehen. Gängige Data Augmentation Techniken für Bilder sind das Rotieren, Spiegeln, Verzerren und das Ändern der Helligkeit oder des Kontrasts.

In [ ]:
bears = bears.new(item_tfms=RandomResizedCrop(224, min_scale=0.5), batch_tfms=aug_transforms(mult=2))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

# Modell trainieren

Diesmal nutzen wir ein Resnet18-Modell zum finetunen.

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

# Ergebnisse anzeigen und interpretieren

`learn.show_results()` zeigt Beispielbilder und deren Klassifizierung an (rot bedeutet, dass das Modell das Bild falsch klassifiziert hat).

In [ ]:
learn.show_results()

Um genauer zu untersuchen wo das Modell Fehler macht, kann man sich die Bilder mit dem höchsten `loss` anzeigen lassen. Ein hohes `loss` Bedeutet, dass ein Bild entweder Falsch klassifiziert wurde, oder das Modell sich bei der richtigen Klassifizierung unsicher war. Bei den folgenden Bildern hatte das Modell also Schwierigkeiten eine korrekte Entscheidung zu treffen.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))

Mit der `Confusion Matrix` sieht man, wieviele Bilder "verwechselt" also falsch Klassifiziert wurden.

In [ ]:
interp.plot_confusion_matrix()

# Bereinigen der Daten

An den top_losses erkennt man, dass sich noch einige Fehlerhafte Daten im Datensatz befinden, wodurch die Genauigkeit des Models noch beeinflusst wird. Die ImageScraper Bibliothek enthält einen cleaner, der es uns erlaubt unseren Datensatz zu bereinigen.

Bereinige den Datensatz und starte erneut bei der Erzeugung des DataBlocks. Du wirst sehen, dass die Genauigkeit des Models jetzt zunehmen wird. 

In [ ]:
from jmd_imagescraper.imagecleaner import *

display_image_cleaner(path)